In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [1]:
!pip uninstall -y tensorflow tensorflow-intel tf-nightly
!pip install tensorflow==2.16.1

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_nightly 2.21.0.dev20250916
Uninstalling tf_nightly-2.21.0.dev20250916:
  Successfully uninstalled tf_nightly-2.21.0.dev20250916
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 120.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
  

In [1]:
import tensorflow as tf
print(tf.__version__)  # debería mostrar 2.16.1

2.16.1


In [2]:
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-09-20 06:40:44--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-09-20 06:40:45 (10.9 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-09-20 06:40:45--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [4]:
# Reading the data with pandas
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

In [5]:
train_df.head()

,label,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [6]:
test_df.head()

,label,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [7]:
train_df.shape

(4179, 2)

In [8]:
test_df.shape

(1392, 2)

In [9]:
# Seeing the number of label in train_df
train_df.label.value_counts()

,count
label,
ham,3619
spam,560


In [10]:
# Seeing the number of label in test_df
test_df.label.value_counts()

,count
label,
ham,1205
spam,187


In [11]:
# Preprocess labels (ham=0, spam=1)
train_labels = train_df['label'].map({'ham': 0, 'spam': 1}).values
test_labels = test_df['label'].map({'ham': 0, 'spam': 1}).values

In [12]:
# Preprocess text data. Hyperparameters
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

In [13]:
# Create TextVectorization layer\n",
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
    )

vectorizer.adapt(train_df['message'].values)

In [14]:
# Convert text to sequences
train_sequences = vectorizer(train_df['message'].values)
test_sequences = vectorizer(test_df['message'].values)

In [15]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_sequences, test_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

In [16]:
# Build the model
def build_model():

  model = keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 64, input_length=MAX_SEQUENCE_LENGTH),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
      tf.keras.layers.GlobalMaxPool1D(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])

  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
      )

  return model



In [17]:
model = build_model()
model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))  # batch_size = None
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 714,369 (2.73 MB)

 Trainable params: 714,369 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    verbose=1
    )

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 17s 102ms/step - accuracy: 0.8883 - loss: 0.3477 - val_accuracy: 0.9634 - val_loss: 0.1490
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 98ms/step - accuracy: 0.9780 - loss: 0.0986 - val_accuracy: 0.9720 - val_loss: 0.1185
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - accuracy: 0.9883 - loss: 0.0576 - val_accuracy: 0.9813 - val_loss: 0.0811
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step - accuracy: 0.9933 - loss: 0.0297 - val_accuracy: 0.9799 - val_loss: 0.0993
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - accuracy: 0.9964 - loss: 0.0144 - val_accuracy: 0.9842 - val_loss: 0.0694
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 0.9990 - loss: 0.0039 - val_accuracy: 0.9856 - val_loss: 0.0895
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 13s 98ms/step - accuracy: 0.9998 - loss: 0.0014 - val_accuracy: 0.9856 - val_loss: 0.1053
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 98ms/step - accuracy: 0.9998 - loss: 0.0012 

In [19]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Convert text to sequence
  sequence = vectorizer([pred_text])

  # Get probability of spam (1)
  prob = model.predict(sequence, verbose=0)[0][0]

  # Determine label based on probability threshold
  label = 'spam' if prob >= 0.5 else 'ham'

  return [prob, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[6.5593827e-06, 'ham']


In [20]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
